In [44]:
from pydantic import BaseModel
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [45]:
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import set_global_service_context
from llama_index import GPTVectorStoreIndex
from llama_index import SimpleDirectoryReader
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index import StorageContext, load_index_from_storage

In [46]:
import openai
from Utilities.envVars import *
import os

# Set OpenAI API key and endpoint
azure_endpoint  = f"{OpenAiEndPoint}"
api_key = OpenAiKey
api_version = OpenAiVersion

AZURE_OPENAI_API_KEY="bac5157f2ae544a6ae573bc6de3a62d7"
AZURE_OPENAI_ENDPOINT="https://dataaioaice.openai.azure.com/"
OPENAI_API_VERSION="2023-07-01-preview"

embeddingModelType = "azureopenai"
temperature = 0.3
tokenLength = 1000

if (embeddingModelType == 'azureopenai'):
        llm = AzureOpenAI(
                model="gpt-35-turbo-16k",
                deployment_name=OpenAiChat16k,
                api_key=AZURE_OPENAI_API_KEY,
                azure_endpoint=AZURE_OPENAI_ENDPOINT,
                api_version=OPENAI_API_VERSION,
                )
        embeddings = AzureOpenAIEmbedding(
                model="text-embedding-ada-002",
                deployment_name=OpenAiEmbedding,
                api_key=AZURE_OPENAI_API_KEY,
                azure_endpoint=AZURE_OPENAI_ENDPOINT,
                api_version=OPENAI_API_VERSION,
                )
        logging.info("LLM Setup done")
elif embeddingModelType == "openai":
       print ("OpenAI")

In [47]:
# set up Azure Cognitive Search
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

SearchEndPoint = f"https://{SearchService}.search.windows.net"

# Index name to use
indexName = "edgardatavector"

# Use index client to demonstrate creating an index
indexClient = SearchIndexClient(
    endpoint=SearchEndPoint,
    credential=AzureKeyCredential(SearchKey),
)
# Use search client to demonstration using existing index
searchClient = SearchClient(
    endpoint=SearchEndPoint,
    index_name=indexName,
    credential=AzureKeyCredential(SearchKey),
)

In [48]:
# searchClient1 = SearchClient(endpoint=f"https://{SearchService}.search.windows.net",
#         index_name="edgardata",
#         credential=AzureKeyCredential(SearchKey))
# r = searchClient1.search(  
#         search_text="",
#         filter="cik eq '" + "72971" + "' and filingType eq '" + "10-K" + "' and filingYear eq '" + "2023" + "'",
#         select=['id', 'cik', 'company', 'filingType', 'filingDate',
#                                                                                                             'periodOfReport', 'sic', 'stateOfInc', 'fiscalYearEnd',
#                                                                                                             'filingHtmlIndex', 'htmFilingLink', 'completeTextFilingLink',
#                                                                                                             'item1', 'item1A', 'item1B', 'item2', 'item3', 'item4', 'item5',
#                                                                                                             'item6', 'item7', 'item7A', 'item8', 'item9', 'item9A', 'item9B',
#                                                                                                             'item10', 'item11', 'item12', 'item13', 'item14', 'item15',
#                                                                                                             'sourcefile'],
#         semantic_configuration_name="semanticConfig",
#         include_total_count=True
#     )

In [49]:
# for filing in r:
#     lastSecData = filing['item1'] + '\n' + filing['item1A'] + '\n' + filing['item1B'] + '\n' + filing['item2'] + '\n' + filing['item3'] + '\n' + filing['item4'] + '\n' + \
#                     filing['item5'] + '\n' + filing['item6'] + '\n' + filing['item7'] + '\n' + filing['item7A'] + '\n' + filing['item8'] + '\n' + \
#                     filing['item9'] + '\n' + filing['item9A'] + '\n' + filing['item9B'] + '\n' + filing['item10'] + '\n' + filing['item11'] + '\n' + filing['item12'] + '\n' + \
#                     filing['item13'] + '\n' + filing['item14'] + '\n' + filing['item15']

In [7]:
# from llama_index import Document
# documents = []
# documents.append(Document(text=lastSecData, metadata={"symbol": "WFC", "cik": "72971", "filingYear": "2023", "filingType": "10-K"}))

In [50]:
from azure.search.documents import SearchClient
from llama_index.vector_stores import CognitiveSearchVectorStore
from llama_index.vector_stores.cogsearch import (
    IndexManagement,
    MetadataIndexFieldType,
    CognitiveSearchVectorStore,
)

# Example of a complex mapping, metadata field 'theme' is mapped to a differently name index field 'topic' with its type explicitly set
metadata_fields = {
    "symbol": "symbol",
    "cik": "cik",
    "filingYear": "filingYear",
    "filingType": "filingType",
}

vectorStore = CognitiveSearchVectorStore(
    search_or_index_client=indexClient,
    index_name=indexName,
    filterable_metadata_field_keys=metadata_fields,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="content",
    embedding_field_key="contentVector",
    metadata_string_field_key="jsonMetadata",
    doc_id_field_key="docId",
)

In [51]:
from llama_index.text_splitter import SentenceSplitter
sentenceSplitter = SentenceSplitter(chunk_size=4000, chunk_overlap=1000)
serviceContext = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embeddings,
        node_parser=None,
        text_splitter=sentenceSplitter,
)
storageContext = StorageContext.from_defaults(vector_store=vectorStore)

In [45]:
# qaIndex = VectorStoreIndex.from_documents(
#     documents, storage_context=storageContext, service_context=serviceContext
# )

#### Load existing documents from Index otherwise

In [52]:
qaIndex = VectorStoreIndex.from_documents(
    [], storage_context=storageContext, service_context=serviceContext
)

In [68]:
queryEngine = qaIndex.as_query_engine()
response = queryEngine.query("Is there any mentioned on impact due to Covid 19?")
response

Response(response='Yes, there is mention of the impact of the COVID-19 pandemic in the provided context information. It states that the pandemic has led to economic issues, such as declining or adverse economic conditions, reduced business activity, and changes in consumer and business behavior. These factors have affected the demand for products and services, the ability of borrowers to repay loans, and overall financial market performance. The context also mentions that the pandemic has resulted in increased regulatory oversight, changes in law or governmental policy, and the implementation of financial assistance or relief programs. Additionally, it highlights the risks associated with reduced workforces, supply chain disruptions, and changes to the labor market.', source_nodes=[NodeWithScore(node=TextNode(id_='5e3d20ef-ea31-4f7e-baf3-d2179b407a5a', embedding=None, metadata={'symbol': 'PNC', 'cik': '713676', 'filingYear': '2022', 'filingType': '10-K'}, excluded_embed_metadata_keys=[

#### Get another document 

In [54]:
# searchClient1 = SearchClient(endpoint=f"https://{SearchService}.search.windows.net",
#         index_name="edgardata",
#         credential=AzureKeyCredential(SearchKey))
# r = searchClient1.search(  
#         search_text="",
#         filter="cik eq '" + "713676" + "' and filingType eq '" + "10-K" + "' and filingYear eq '" + "2023" + "'",
#         select=['id', 'cik', 'company', 'filingType', 'filingDate',
#                                                                                                             'periodOfReport', 'sic', 'stateOfInc', 'fiscalYearEnd',
#                                                                                                             'filingHtmlIndex', 'htmFilingLink', 'completeTextFilingLink',
#                                                                                                             'item1', 'item1A', 'item1B', 'item2', 'item3', 'item4', 'item5',
#                                                                                                             'item6', 'item7', 'item7A', 'item8', 'item9', 'item9A', 'item9B',
#                                                                                                             'item10', 'item11', 'item12', 'item13', 'item14', 'item15',
#                                                                                                             'sourcefile'],
#         semantic_configuration_name="semanticConfig",
#         include_total_count=True
#     )

In [55]:
# for filing in r:
#     lastSecData = filing['item1'] + '\n' + filing['item1A'] + '\n' + filing['item1B'] + '\n' + filing['item2'] + '\n' + filing['item3'] + '\n' + filing['item4'] + '\n' + \
#                     filing['item5'] + '\n' + filing['item6'] + '\n' + filing['item7'] + '\n' + filing['item7A'] + '\n' + filing['item8'] + '\n' + \
#                     filing['item9'] + '\n' + filing['item9A'] + '\n' + filing['item9B'] + '\n' + filing['item10'] + '\n' + filing['item11'] + '\n' + filing['item12'] + '\n' + \
#                     filing['item13'] + '\n' + filing['item14'] + '\n' + filing['item15']

In [56]:
# from llama_index import Document
# documents = []
# documents.append(Document(text=lastSecData, metadata={"symbol": "PNC", "cik": "713676", "filingYear": "2023", "filingType": "10-K"}))

In [57]:
# qaIndex = VectorStoreIndex.from_documents(
#     documents, storage_context=storageContext, service_context=serviceContext
# )

In [58]:
# queryEngine = qaIndex.as_query_engine()
# response = queryEngine.query("Is there any mentioned on impact due to Covid 19?")
# response

### Create subQuery Engine

In [59]:
from llama_index.schema import TextNode
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

pncFilters = MetadataFilters(
    filters=[ExactMatchFilter(key="symbol", value="PNC"), 
             ExactMatchFilter(key="filingYear", value="2023"), 
             ExactMatchFilter(key="filingType", value="10-K")]
)
wfcFilters = MetadataFilters(
    filters=[ExactMatchFilter(key="symbol", value="WFC"), 
             ExactMatchFilter(key="filingYear", value="2023"), 
             ExactMatchFilter(key="filingType", value="10-K")]
)
wfcEng = qaIndex.as_query_engine(filters=wfcFilters, similarity_top_k=3)
pncEng = qaIndex.as_query_engine(filters=pncFilters, similarity_top_k=3)

# pncRetriever = qaIndex.as_retriever(filters=pncFilters)
# wfcRetriever = qaIndex.as_retriever(filters=wfcFilters)

# from llama_index.query_engine import RetrieverQueryEngine
# from llama_index.query_engine import BaseQueryEngine

# pncEngine = RetrieverQueryEngine(
#     retriever=pncRetriever,
# )
# wfcEngine = RetrieverQueryEngine(
#     retriever=wfcRetriever,
# )

In [79]:
question = "Summarize the future growth and business risk"

In [73]:
response = pncEng.query(question)
print(response.response)

The future growth and business risk for PNC are influenced by various factors. In terms of future growth, PNC's financial performance is impacted significantly by market interest rates and movements in those rates. Changes in interest rates can affect net interest income, the ability of borrowers to meet obligations, demand for interest rate-based products and services, and the effectiveness of hedging strategies. Higher interest rates can enhance net interest income but may also decrease the value of fixed-rate financial instruments and impact the value of asset classes such as real estate. On the other hand, lower interest rates can have a negative impact on net interest margin and net interest income. PNC's growth is also influenced by the creditworthiness of its customers and counterparties. Credit risk is inherent in the financial services business, and PNC manages credit risk through assessing and monitoring the creditworthiness of customers, diversifying its loan portfolio, and 

In [74]:
wfcEng.query(question)

Response(response="The future growth of Wells Fargo & Company is subject to various factors and risks. The company operates in a highly competitive financial services industry and faces competition from banks, nonbank institutions, and technological advancements. Changes in regulations and legislation could restrict the company's ability to diversify, make acquisitions, and pay dividends. The company's growth is also influenced by economic, market, and political conditions. Additionally, the company is committed to promoting diversity, equity, and inclusion and has implemented various programs and initiatives in this regard. However, the company is also exposed to risks such as credit risk, market risk, operational risk, and regulatory risk. The company's financial performance and ability to grow are dependent on its ability to effectively manage these risks and adapt to changing market conditions.", source_nodes=[NodeWithScore(node=TextNode(id_='f40c05a7-86d7-449d-8d9a-d3248f2a02e0', 

In [25]:
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
        QueryEngineTool(
            query_engine=pncEng, 
            metadata=ToolMetadata(name='2023PNC_10k', description='Provides information about PNC financials for year 2023')
        ),
        QueryEngineTool(
            query_engine=wfcEng, 
            metadata=ToolMetadata(name='2023WFC_10k', description='Provides information about WFC financials for year 2023')
        )
]

In [26]:
subQueryEngine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=serviceContext,
)

In [27]:
# enable INFO level logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [28]:
async def queryData(query, subQueryEngine):
    #queries the engine with the input text    
    response = await subQueryEngine.aquery(query)
    return response.response

In [29]:
response = await queryData(question, subQueryEngine)
response

Generated 4 sub questions.
[2023PNC_10k] Q: What is the projected revenue growth for PNC in 2023?
[2023WFC_10k] Q: What is the projected revenue growth for WFC in 2023?
[2023PNC_10k] Q: What is the business risk assessment for PNC in 2023?
[2023WFC_10k] Q: What is the business risk assessment for WFC in 2023?
[2023PNC_10k] A: PNC projects a revenue growth of 6% to 8% for the full year 2023 compared to the full year 2022.
[2023WFC_10k] A: The provided context information does not include a specific business risk assessment for WFC in 2023.
[2023WFC_10k] A: The projected revenue growth for WFC in 2023 is not provided in the given context information.
[2023PNC_10k] A: The business risk assessment for PNC in 2023 includes several key risks. One of the significant risks is the risk associated with the creditworthiness of customers and counterparties. This risk arises from extending credit, purchasing securities, and entering into financial derivative transactions. PNC manages this risk by a

'The future growth for PNC in 2023 is projected to be between 6% and 8% compared to the previous year. However, the projected revenue growth for WFC in 2023 is not provided in the given context information.\n\nIn terms of business risk assessment, PNC faces several key risks in 2023. These include creditworthiness risk, information system failures or breaches, risks related to interest rate fluctuations and market conditions, and operational risks such as technology and third-party breaches of data security. PNC has implemented measures to manage and mitigate these risks, including assessing and monitoring customer creditworthiness, diversifying the loan portfolio, implementing cybersecurity measures, and managing interest rate risk through asset and liability management practices.\n\nOn the other hand, the specific business risk assessment for WFC in 2023 is not mentioned in the provided context information.'

In [42]:
from llama_index.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(use_async=True, service_context=serviceContext)
summarizeTopicPrompt = """You are an AI assistant tasked with summarizing documents from 
        earning call transcripts, annual reports, SEC filings and financial statements like income statement, cashflow and 
        balance sheets. Additionally you may also be asked to answer questions about financial ratios and other financial metrics.
        Your summary should accurately capture the key information in the document while avoiding the omission of any domain-specific words. 
        Please generate a concise and comprehensive summary that includes details. 
        Ensure that the summary is easy to understand and provides an accurate representation. 
        Begin the summary with a brief introduction, followed by the main points.
        Generate the summary with minimum of 7 paragraphs and maximum of 10 paragraphs.
        Please remember to use clear language and maintain the integrity of the original information without missing any important details:"""
sectionTopicPrompt = """You are an AI assistant tasked with summarizing sections from the financial document like 10-K and 10-Q report. 
            Your summary should accurately capture the key information in the document while avoiding the omission of any domain-specific words. 
            Please remember to use clear language and maintain the integrity of the original information without missing any important details.
            Please generate a concise and comprehensive 3 paragraphs summary of the following document. 
            Ensure that the summary is generated for each of the following sections:"""
nodes = wfcEng.retrieve("Management Negative Sentiment")
textChunks = []
# for node in nodes:
#     textChunks.append(node.text)
textChunks.append(filing['item1'])
summary = summarizer.get_response(sectionTopicPrompt, textChunks)
summary

"Wells Fargo & Company is a financial holding company and bank holding company with assets of approximately $1.9 trillion as of December 31, 2022. The company provides a wide range of banking, investment, and mortgage products and services to individuals, businesses, and institutions in the United States and internationally. Its subsidiaries offer consumer financial products and services, including checking and savings accounts, credit and debit cards, and various types of loans. Wells Fargo also provides financial solutions to businesses through commercial loans, trade financing, and investment banking services. The company has a strong focus on human capital, investing in its employees through competitive compensation, career development opportunities, and work-life programs.\n\nWells Fargo operates under the regulatory framework applicable to bank holding companies and is subject to supervision and examination by various federal and state regulatory agencies. The company is required

In [76]:
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

companies = ['PNC', 'WFC']
years = ['2023']
filingType = ['10-K']
queryEngineTools = []
for company in companies:
    for year in years:
        for filing in filingType:
            filters = MetadataFilters(
                filters=[ExactMatchFilter(key="symbol", value=company), 
                        ExactMatchFilter(key="filingYear", value=year), 
                        ExactMatchFilter(key="filingType", value=filing)]
            )
            engine = qaIndex.as_query_engine(filters=filters, similarity_top_k=3)
            queryEngineTools.append(
                QueryEngineTool(query_engine=engine, 
                                metadata=ToolMetadata(
                                    name=f'{year}{company}_{filingType}', 
                                    description=f'Provides information about {company} for year {year} and filing type {filingType}')
                )
            )

In [78]:
subQueryEngine1 = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=queryEngineTools,
    service_context=serviceContext,
)

In [87]:
response = await queryData(question, subQueryEngine1)

#### Use Existing Index

In [89]:
def createSecFilingsVectorLlamaIndex(SearchService, SearchKey, indexName):
    indexClient = SearchIndexClient(endpoint=f"https://{SearchService}.search.windows.net/",
        credential=AzureKeyCredential(SearchKey))

    metadata_fields = {
        "symbol": "symbol",
        "cik": "cik",
        "filingYear": "filingYear",
        "filingType": "filingType",
    }
    vectorStore = CognitiveSearchVectorStore(
        search_or_index_client=indexClient,
        index_name=indexName,
        filterable_metadata_field_keys=metadata_fields,
        index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
        id_field_key="id",
        chunk_field_key="content",
        embedding_field_key="contentVector",
        metadata_string_field_key="jsonMetadata",
        doc_id_field_key="docId",
    )
    return vectorStore

In [94]:
import asyncio

secFilingsVectorIndexName = 'edgardatavector'
vectorStore = createSecFilingsVectorLlamaIndex(SearchService, SearchKey, secFilingsVectorIndexName)
storageContext = StorageContext.from_defaults(vector_store=vectorStore)
sentenceSplitter = SentenceSplitter(chunk_size=4000, chunk_overlap=1000)
serviceContext = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embeddings,
        node_parser=None,
        text_splitter=sentenceSplitter,
)

In [105]:
selectedCompanies = ['JPM', 'BAC', 'GS']
selectedYears = ['2022','2021', '2020']
filingType = '10-K'
queryEngineTools = []
for company in selectedCompanies:
    for filingYear in selectedYears:
        try:
            # Create the filter to be used for the Query Engine
            llamaIndex = VectorStoreIndex.from_documents([], storage_context=storageContext, service_context=serviceContext)
            vectorFilters = MetadataFilters(
                filters=[ExactMatchFilter(key="symbol", value=company), 
                        ExactMatchFilter(key="filingYear", value=filingYear), 
                        ExactMatchFilter(key="filingType", value=filingType)]
            )
            engine = llamaIndex.as_query_engine(filters=vectorFilters, similarity_top_k=3)
            queryEngineTools.append(
                QueryEngineTool(query_engine=engine, 
                                metadata=ToolMetadata(
                                    name=f'{filingYear}{company}_{filingType}', 
                                    description=f'Provides information from financial document like 10-K and 10-Q on {company} for {filingYear}') 
                )
            )
        except Exception as e:
            logging.info("Error in processing Step 3 : " + str(e))

In [106]:
subQueryEngine = SubQuestionQueryEngine.from_defaults(
        query_engine_tools=queryEngineTools,
        service_context=serviceContext,
        )

In [107]:
async def subQueryEngineExecute(query, subQueryEngine):
    response = await subQueryEngine.aquery(query)
    return response.response

In [108]:
question = 'What is the sales and trading revenue for equities for all year across all companies.  Display the output as Table with columns as company, year and revenue(figures in million)'
response = await subQueryEngineExecute(question, subQueryEngine)

Generated 9 sub questions.
[2022JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2022?
[2021JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2021?
[2020JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2020?
[2022BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2022?
[2021BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2021?
[2020BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2020?
[2022GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2022?
[2021GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2021?
[2020GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2020?
[2020BAC_10-K] A: The sales and trading revenue for equities for BAC in 2020 is not provided in the given context information.
[2022GS_10-K] A: The sales and trading revenue for equities for GS in 2022 is 

#### Let's test it out on default Chunk Size & Chunk Overlap
#### Because on 1st attempt it didn't work to build index from "SEC Extracted data", 2nd Attempt is loading data from PDF

In [158]:
sentenceSplitter = SentenceSplitter(chunk_size=4000, chunk_overlap=1000)
serviceContext = ServiceContext.from_defaults(
            llm=llm,
            embed_model=embeddings,
            node_parser=None,
            text_splitter=sentenceSplitter,
)

#### 2nd Attempt - Load BofA Docs Manually from PDF

In [169]:
bofA2018Docs = SimpleDirectoryReader(input_files=["data/BOFA/BofA 2018.pdf"]).load_data()
for doc in bofA2018Docs:
    doc.metadata = {"symbol": "BAC", "cik": "70858", "filingYear": "2018", "filingType": "10-K"}

bofA2019Docs = SimpleDirectoryReader(input_files=["data/BOFA/BofA 2019.pdf"]).load_data()
for doc in bofA2019Docs:
    doc.metadata = {"symbol": "BAC", "cik": "70858", "filingYear": "2019", "filingType": "10-K"}

bofA2020Docs = SimpleDirectoryReader(input_files=["data/BOFA/BofA 2020.pdf"]).load_data()
for doc in bofA2020Docs:
    doc.metadata = {"symbol": "BAC", "cik": "70858", "filingYear": "2020", "filingType": "10-K"}

bofA2021Docs = SimpleDirectoryReader(input_files=["data/BOFA/BofA 2021.pdf"]).load_data()
for doc in bofA2021Docs:
    doc.metadata = {"symbol": "BAC", "cik": "70858", "filingYear": "2021", "filingType": "10-K"}

bofA2022Docs = SimpleDirectoryReader(input_files=["data/BOFA/BofA 2022.pdf"]).load_data()
for doc in bofA2022Docs:
    doc.metadata = {"symbol": "BAC", "cik": "70858", "filingYear": "2022", "filingType": "10-K"}

#### 2nd Attempt - Load JPMC Docs Manually from PDF

In [174]:
JPMC2018Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2018.pdf"]).load_data()
for doc in JPMC2018Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2018", "filingType": "10-K"}

JPMC2019Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2019.pdf"]).load_data()
for doc in JPMC2019Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2019", "filingType": "10-K"}

JPMC2020Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2020.pdf"]).load_data()
for doc in JPMC2020Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2020", "filingType": "10-K"}

JPMC2021Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2021.pdf"]).load_data()
for doc in JPMC2021Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2021", "filingType": "10-K"}

JPMC2022Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2022.pdf"]).load_data()
for doc in JPMC2022Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2022", "filingType": "10-K"}

In [176]:
JPMC2020Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2020.pdf"]).load_data()
for doc in JPMC2020Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2020", "filingType": "10-K"}

In [177]:
JPMC2021Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2021.pdf"]).load_data()
for doc in JPMC2021Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2021", "filingType": "10-K"}

In [178]:
JPMC2022Docs = SimpleDirectoryReader(input_files=["data/JPMC/JPMC2022.pdf"]).load_data()
for doc in JPMC2022Docs:
    doc.metadata = {"symbol": "JPM", "cik": "19617", "filingYear": "2022", "filingType": "10-K"}

#### 2nd Attempt - Load GS Docs Manually from PDF

In [179]:
GS2018Docs = SimpleDirectoryReader(input_files=["data/GS/GS2018.pdf"]).load_data()
for doc in GS2018Docs:
    doc.metadata = {"symbol": "GS", "cik": "886982", "filingYear": "2018", "filingType": "10-K"}

In [180]:
GS2019Docs = SimpleDirectoryReader(input_files=["data/GS/GS2019.pdf"]).load_data()
for doc in GS2019Docs:
    doc.metadata = {"symbol": "GS", "cik": "886982", "filingYear": "2019", "filingType": "10-K"}

In [181]:
GS2020Docs = SimpleDirectoryReader(input_files=["data/GS/GS2020.pdf"]).load_data()
for doc in GS2020Docs:
    doc.metadata = {"symbol": "GS", "cik": "886982", "filingYear": "2020", "filingType": "10-K"}

In [182]:
GS2021Docs = SimpleDirectoryReader(input_files=["data/GS/GS2021.pdf"]).load_data()
for doc in GS2021Docs:
    doc.metadata = {"symbol": "GS", "cik": "886982", "filingYear": "2021", "filingType": "10-K"}

In [183]:
GS2022Docs = SimpleDirectoryReader(input_files=["data/GS/GS2022.pdf"]).load_data()
for doc in GS2022Docs:
    doc.metadata = {"symbol": "GS", "cik": "886982", "filingYear": "2022", "filingType": "10-K"}

In [184]:
import requests
import typing
import logging
from llama_index import VectorStoreIndex, ServiceContext, StorageContext, Document
CONNECT_TIMEOUT = 5
READ_TIMEOUT = 30
BASE_URL_v3: str = "https://financialmodelingprep.com/api/v3/"
BASE_URL_v4: str = "https://financialmodelingprep.com/api/v4/"
DEFAULT_LIMIT: int = 10

In [185]:
def __return_json_v4(
    path: str, query_vars: typing.Dict
) -> typing.Optional[typing.List]:
    """
    Query URL for JSON response for v4 of FMP API.

    :param path: Path after TLD of URL
    :param query_vars: Dictionary of query values (after "?" of URL)
    :return: JSON response
    """
    url = f"{BASE_URL_v4}{path}"
    return_var = None
    try:
        response = requests.get(
            url, params=query_vars, timeout=(CONNECT_TIMEOUT, READ_TIMEOUT)
        )
        if len(response.content) > 0:
            return_var = response.json()

        if len(response.content) == 0 or (
            isinstance(return_var, dict) and len(return_var.keys()) == 0
        ):
            logging.warning("Response appears to have no data.  Returning empty List.")
            return_var = []

    except requests.Timeout:
        logging.error(f"Connection to {url} timed out.")
    except requests.ConnectionError:
        logging.error(
            f"Connection to {url} failed:  DNS failure, refused connection or some other connection related "
            f"issue."
        )
    except requests.TooManyRedirects:
        logging.error(
            f"Request to {url} exceeds the maximum number of predefined redirections."
        )
    except Exception as e:
        logging.error(
            f"A requests exception has occurred that we have not yet detailed an 'except' clause for.  "
            f"Error: {e}"
        )
    return return_var


In [186]:
def searchCik(
    apikey: str,
    ticker: str,
) -> typing.Optional[typing.List[typing.Dict]]:
    """
    Query FMP /mapper-cik-company/ API.

    Company CIK mapper

    :param apikey: Your API key.
    :param ticker: String of name.
    :return: A list of dictionaries.
    """
    path = f"mapper-cik-company/{ticker}"
    query_vars = {"apikey": apikey}
    return __return_json_v4(path=path, query_vars=query_vars)

In [187]:
def findSecFiling(SearchService, SearchKey, indexName, cik, filingType, filingYear, returnFields=["id", "content", "sourcefile"] ):
    searchClient = SearchClient(endpoint=f"https://{SearchService}.search.windows.net",
        index_name=indexName,
        credential=AzureKeyCredential(SearchKey))
    
    try:
        r = searchClient.search(  
            search_text="",
            filter="cik eq '" + cik + "' and filingType eq '" + filingType + "' and filingYear eq '" + filingYear + "'",
            select=returnFields,
            semantic_configuration_name="semanticConfig",
            include_total_count=True
        )
        return r
    except Exception as e:
        logging.info(e)

    return None

In [188]:
def indexSecFilingsSectionsLlama(vectorStore, serviceContext, secData, cik, symbol, filingYear, filingType):
    documents = []
    documents.append(Document(text=secData, metadata={"symbol": symbol, "cik": cik, "filingYear": filingYear, "filingType": filingType}))
    storageContext = StorageContext.from_defaults(vector_store=vectorStore)
    secIndex = VectorStoreIndex.from_documents(documents, storage_context=storageContext, service_context=serviceContext)

In [189]:
def findSecVectorFilings(SearchService, SearchKey, indexName, cik, symbol, filingYear, filingType, returnFields=["id", "content", "sourcefile"] ):
    searchClient = SearchClient(endpoint=f"https://{SearchService}.search.windows.net",
        index_name=indexName,
        credential=AzureKeyCredential(SearchKey))
    
    try:
        r = searchClient.search(  
            search_text="",
            filter="cik eq '" + cik + "' and symbol eq '" + symbol + "' and filingYear eq '" + filingYear  + "' and filingType eq '" + filingType + "'",
            select=returnFields,
            semantic_configuration_name="semanticConfig",
            include_total_count=True
        )
        return r
    except Exception as e:
        logging.info(e)

    return None

#### Load from the PDF Docs (Attempt #2)

In [190]:
secFilingsVectorIndexName = 'edgartempvector'
vectorStore = createSecFilingsVectorLlamaIndex(SearchService, SearchKey, secFilingsVectorIndexName)
storageContext = StorageContext.from_defaults(vector_store=vectorStore)
sentenceSplitter = SentenceSplitter(chunk_size=4000, chunk_overlap=1000)
serviceContext = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embeddings,
        node_parser=None,
        text_splitter=sentenceSplitter,
)

In [191]:
VectorStoreIndex.from_documents(bofA2018Docs, storage_context=storageContext, service_context=serviceContext)

In [192]:
VectorStoreIndex.from_documents(bofA2019Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(bofA2020Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(bofA2021Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(bofA2022Docs, storage_context=storageContext, service_context=serviceContext)

In [193]:
VectorStoreIndex.from_documents(JPMC2018Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(JPMC2019Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(JPMC2020Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(JPMC2021Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(JPMC2022Docs, storage_context=storageContext, service_context=serviceContext)

In [194]:
VectorStoreIndex.from_documents(GS2018Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(GS2018Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(GS2020Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(GS2021Docs, storage_context=storageContext, service_context=serviceContext)
VectorStoreIndex.from_documents(GS2022Docs, storage_context=storageContext, service_context=serviceContext)

In [195]:
selectedCompanies = ['JPM', 'BAC', 'GS']
selectedYears = ['2022','2021', '2020']
filingType = '10-K'
query_engine_tools = []
for company in selectedCompanies:
    for filingYear in selectedYears:
        try:
            # Create the filter to be used for the Query Engine
            llamaIndex = VectorStoreIndex.from_documents([], storage_context=storageContext, service_context=serviceContext)
            vectorFilters = MetadataFilters(
                filters=[ExactMatchFilter(key="symbol", value=company), 
                        ExactMatchFilter(key="filingYear", value=filingYear), 
                        ExactMatchFilter(key="filingType", value=filingType)]
            )
            engine = llamaIndex.as_query_engine(filters=vectorFilters)
            query_engine_tools.append(
                QueryEngineTool(query_engine=engine, 
                                metadata=ToolMetadata(
                                    name=f'{filingYear}{company}_{filingType}', 
                                    description=f'Provides information from financial document like 10-K and 10-Q on {company} for {filingYear}') 
                )
            )
        except Exception as e:
            logging.info("Error in processing Step 3 : " + str(e))

In [196]:
subQueryEngine = SubQuestionQueryEngine.from_defaults(
        query_engine_tools=query_engine_tools,
        service_context=serviceContext,
        )

In [197]:
async def subQueryEngineExecute(query, subQueryEngine):
    response = await subQueryEngine.aquery(query)
    return response.response

In [200]:
question = 'What is the sales and trading revenue for equities for all year across all companies.  Display the output as Table with columns as company, year and revenue(figures in million)'
response = await subQueryEngineExecute(question, subQueryEngine)
response

Generated 9 sub questions.
[2022JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2022?
[2021JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2021?
[2020JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2020?
[2022BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2022?
[2021BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2021?
[2020BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2020?
[2022GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2022?
[2021GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2021?
[2020GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2020?
[2020JPM_10-K] A: The sales and trading revenue for equities for JPMorgan Chase & Co. in 2020 was $8.6 billion.
[2022JPM_10-K] A: The sales and trading revenue for equities for JPMorgan Chase & Co. in 2022

'Company | Year | Revenue (in million)\n--------|------|--------------------\nJPM | 2022 | Not provided\nJPM | 2021 | 7,773\nJPM | 2020 | 8,600\nBAC | 2022 | 6,572\nBAC | 2021 | 6,428\nBAC | 2020 | 5,422\nGS | 2022 | 11,720\nGS | 2021 | Not provided\nGS | 2020 | 9,570'

In [201]:
question = 'How differently those banks are handling CCAR.  Give me the answer in bulleted format with breakdown by company and by each year'
response = await subQueryEngineExecute(question, subQueryEngine)
response

Generated 9 sub questions.
[2022JPM_10-K] Q: How did JPM handle CCAR in 2022?
[2021JPM_10-K] Q: How did JPM handle CCAR in 2021?
[2020JPM_10-K] Q: How did JPM handle CCAR in 2020?
[2022BAC_10-K] Q: How did BAC handle CCAR in 2022?
[2021BAC_10-K] Q: How did BAC handle CCAR in 2021?
[2020BAC_10-K] Q: How did BAC handle CCAR in 2020?
[2022GS_10-K] Q: How did GS handle CCAR in 2022?
[2021GS_10-K] Q: How did GS handle CCAR in 2021?
[2020GS_10-K] Q: How did GS handle CCAR in 2020?
[2021JPM_10-K] A: JPMorgan Chase completed the 2021 Comprehensive Capital Analysis and Review (CCAR) stress test process. The Federal Reserve affirmed the Firm's 2021 Stress Capital Buffer (SCB) requirement of 3.2% and the Firm's Standardized CET1 capital ratio requirement of 11.2%. These requirements became effective on October 1, 2021, and will remain in effect until September 30, 2022.
[2020GS_10-K] A: The given context does not provide any information about how GS (Goldman Sachs) handled CCAR (Comprehensive Cap

'- JPMorgan Chase:\n  - 2022: Completed CCAR stress test process, affirmed SCB requirement of 4.0% and CET1 capital ratio requirement of 12.0%.\n  - 2021: Completed CCAR stress test process, affirmed SCB requirement of 3.2% and CET1 capital ratio requirement of 11.2%.\n  - 2020: Completed CCAR stress test process, updated and resubmitted capital plans due to COVID-19 impact, allowed to resume share repurchases in Q1 2021.\n\n- Bank of America (BAC):\n  - 2022: Increased SCB to 3.4% from 2.5%, repurchased $5.1 billion of common stock.\n  - 2021: Renewed $25 billion common stock repurchase program, repurchased $25.1 billion of common stock.\n  - 2020: Integrated CCAR with Basel 3 Final Rule, subject to 2.5% SCB.\n\n- Goldman Sachs (GS):\n  - 2022: FRB reduced SCB based on CCAR submission.\n  - 2021: FRB reduced SCB from 6.6% to 6.4% based on CCAR submission.\n  - 2020: No information provided in the context.'

In [202]:
question = 'Compare and contrast the revenue between 2021 and 2022.  Display the output as JSON object with keys as company, year and revenue'
response = await subQueryEngineExecute(question, subQueryEngine)
response

Generated 6 sub questions.
[2021JPM_10-K] Q: What is the revenue for JPM in 2021?
[2022JPM_10-K] Q: What is the revenue for JPM in 2022?
[2021BAC_10-K] Q: What is the revenue for BAC in 2021?
[2022BAC_10-K] Q: What is the revenue for BAC in 2022?
[2021GS_10-K] Q: What is the revenue for GS in 2021?
[2022GS_10-K] Q: What is the revenue for GS in 2022?
[2021JPM_10-K] A: The revenue for JPM in 2021 is $121,649 million.
[2022JPM_10-K] A: The revenue for JPM in 2022 is $47.9 billion.
[2021BAC_10-K] A: The revenue for BAC in 2021 is $89,113 million.
[2021GS_10-K] A: The revenue for GS in 2021 was $59.34 billion.
[2022GS_10-K] A: The revenue for GS in 2022 was $47.37 billion.
[2022BAC_10-K] A: The revenue for BAC in 2022 is $19,273 million.


'{\n  "JPM": {\n    "2021": "$121,649 million",\n    "2022": "$47.9 billion"\n  },\n  "BAC": {\n    "2021": "$89,113 million",\n    "2022": "$19,273 million"\n  },\n  "GS": {\n    "2021": "$59.34 billion",\n    "2022": "$47.37 billion"\n  }\n}'

#### Following is for Attempt #1

In [ ]:
selectedCompanies = ['JPM', 'BAC', 'GS']
selectedYears = ['2022','2021', '2020']
filingType = '10-K'
secFilingList = []
secFilingIndexName = 'edgardata'
for company in selectedCompanies:
    cik = str(int(searchCik(apikey=FmpKey, ticker=company)[0]["companyCik"]))
    for filingYear in selectedYears:
        try:
            # Create the filter to be used for the Query Engine
            llamaIndex = VectorStoreIndex.from_documents([], storage_context=storageContext, service_context=serviceContext)
            vectorFilters = MetadataFilters(
                filters=[ExactMatchFilter(key="symbol", value=company), 
                        ExactMatchFilter(key="filingYear", value=filingYear), 
                        ExactMatchFilter(key="filingType", value=filingType)]
            )
            r = findSecFiling(SearchService, SearchKey, secFilingIndexName, cik, filingType, filingYear, returnFields=['id', 'cik', 'company', 'filingType', 'filingDate',
                                                                                                'periodOfReport', 'sic', 'stateOfInc', 'fiscalYearEnd',
                                                                                                'filingHtmlIndex', 'htmFilingLink', 'completeTextFilingLink',
                                                                                                'item1', 'item1A', 'item1B', 'item2', 'item3', 'item4', 'item5',
                                                                                                'item6', 'item7', 'item7A', 'item8', 'item9', 'item9A', 'item9B',
                                                                                                'item10', 'item11', 'item12', 'item13', 'item14', 'item15', 'content',
                                                                                                'sourcefile'])
            for filing in r:
                secFilingList.append({
                    "id": filing['id'],
                    "cik": filing['cik'],
                    "company": filing['company'],
                    "filingType": filing['filingType'],
                    "filingDate": filing['filingDate'],
                    "periodOfReport": filing['periodOfReport'],
                    "sic": filing['sic'],
                    "stateOfInc": filing['stateOfInc'],
                    "fiscalYearEnd": filing['fiscalYearEnd'],
                    "filingHtmlIndex": filing['filingHtmlIndex'],
                    "completeTextFilingLink": filing['completeTextFilingLink'],
                    "item1": filing['item1'],
                    "item1A": filing['item1A'],
                    "item1B": filing['item1B'],
                    "item2": filing['item2'],
                    "item3": filing['item3'],
                    "item4": filing['item4'],
                    "item5": filing['item5'],
                    "item6": filing['item6'],
                    "item7": filing['item7'],
                    "item7A": filing['item7A'],
                    "item8": filing['item8'],
                    "item9": filing['item9'],
                    "item9A": filing['item9A'],
                    "item9B": filing['item9B'],
                    "item10": filing['item10'],
                    "item11": filing['item11'],
                    "item12": filing['item12'],
                    "item13": filing['item13'],
                    "item14": filing['item14'],
                    "item15": filing['item15'],
                    "sourcefile": filing['sourcefile']
                })

            secFilingsVectorIndexName = 'edgartempvector'
            #createSecFilingsVectorIndex(SearchService, SearchKey, secFilingsVectorIndexName)
            vectorStore = createSecFilingsVectorLlamaIndex(SearchService, SearchKey, secFilingsVectorIndexName)
            # Now that we got the vectorStore, let's create the index from the documents
            storageContext = StorageContext.from_defaults(vector_store=vectorStore)
            llamaIndex = VectorStoreIndex.from_documents([], storage_context=storageContext, service_context=serviceContext)
            r = findSecVectorFilings(SearchService, SearchKey, secFilingsVectorIndexName, cik, company, filingYear, filingType, returnFields=['id', 'cik', 'symbol', 'filingYear', 'filingType',
                                                                                                                                'content'])

            if r.get_count() == 0:
                    logging.info("Processing SEC Filings for CIK : " + str(cik) + " and Symbol : " + str(company) + " and Year : " + str(filingYear) + " and Filing Type : " + str(filingType))
                    indexSecFilingsSectionsLlama(vectorStore, serviceContext, filing['content'], cik, company, filingYear, filingType)

        except Exception as e:
            logging.info("Error in processing Step 3 : " + str(e))

In [135]:
query_engine_tools = []
for company in selectedCompanies:
    for filingYear in selectedYears:
        try:
            # Create the filter to be used for the Query Engine
            llamaIndex = VectorStoreIndex.from_documents([], storage_context=storageContext, service_context=serviceContext)
            vectorFilters = MetadataFilters(
                filters=[ExactMatchFilter(key="symbol", value=company), 
                        ExactMatchFilter(key="filingYear", value=filingYear), 
                        ExactMatchFilter(key="filingType", value=filingType)]
            )
            engine = llamaIndex.as_query_engine(filters=vectorFilters)
            query_engine_tools.append(
                QueryEngineTool(query_engine=engine, 
                                metadata=ToolMetadata(
                                    name=f'{filingYear}{company}_{filingType}', 
                                    description=f'Provides information from financial document like 10-K and 10-Q on {company} for {filingYear}') 
                )
            )
        except Exception as e:
            logging.info("Error in processing Step 3 : " + str(e))

In [136]:
subQueryEngine = SubQuestionQueryEngine.from_defaults(
        query_engine_tools=query_engine_tools,
        service_context=serviceContext,
        )

In [137]:
async def subQueryEngineExecute(query, subQueryEngine):
    response = await subQueryEngine.aquery(query)
    return response.response

In [138]:
question = 'What is the sales and trading revenue for equities for all year across all companies.  Display the output as Table with columns as company, year and revenue(figures in million)'
response = await subQueryEngineExecute(question, subQueryEngine)

Generated 9 sub questions.
[2022JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2022?
[2021JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2021?
[2020JPM_10-K] Q: What is the sales and trading revenue for equities for JPM in 2020?
[2022BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2022?
[2021BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2021?
[2020BAC_10-K] Q: What is the sales and trading revenue for equities for BAC in 2020?
[2022GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2022?
[2021GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2021?
[2020GS_10-K] Q: What is the sales and trading revenue for equities for GS in 2020?
[2020BAC_10-K] A: The sales and trading revenue for equities for BAC in 2020 is not provided in the given context information.
[2021GS_10-K] A: I'm sorry, but I cannot provide the sales and trading revenue

In [156]:
response = query_engine_tools[3].query_engine.query("What is the sales and trading revenue for equities.  Display the output as Table with columns as company, year and revenue(figures in million)")
response.response

'Company | Year | Revenue (in millions)\n--------|------|---------------------\nBAC     | 2021 | 6,428\nBAC     | 2020 | 5,422'